# เชียงราย

In [ ]:
!pip -q install earthengine-api geemap google-cloud-storage google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

import ee, geemap
ee.Authenticate()
ee.Initialize(project='*******')


In [ ]:
import ee, datetime
# AOI: เชียงราย
aoi = ee.Geometry.Rectangle([98.9, 18.9, 99.9, 19.7])

start_date = '2025-02-01'
end_date   = '2025-05-15'


In [ ]:
worldcover = ee.Image('ESA/WorldCover/v200/2021').eq(40)
dynworld   = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
              .filterDate(start_date, end_date)
              .filterBounds(aoi)
              .select('label').mode().eq(1))
cropland = worldcover.Or(dynworld).selfMask()

In [ ]:
def s2_clean(start, end, geom):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate(start, end).filterBounds(geom))
    def mask_cloud(i):
        qa = i.select('QA60')
        cloud = qa.bitwiseAnd(1<<10).Or(qa.bitwiseAnd(1<<11))
        return i.updateMask(cloud.Not()).divide(10000)
    return s2.map(mask_cloud).median().clip(geom)

def nbr(img):
    return img.normalizedDifference(['B8','B12']).rename('NBR')

pre  = s2_clean('2025-01-15','2025-01-31', aoi)
post = s2_clean(start_date, end_date, aoi)

dNBR = nbr(pre).subtract(nbr(post)).rename('dNBR').updateMask(cropland)
burn  = dNBR.gt(0.30).selfMask()
heavy = dNBR.gt(0.50).selfMask()


In [ ]:
scale = 20
polys = (burn.reduceToVectors(
            geometry=aoi, scale=scale, geometryType='polygon',
            labelProperty='burn', maxPixels=1e13)
         .map(lambda f: f.set({
             'date': end_date,
             'class': 'burn',
             'area_ha': f.geometry().area(1).divide(10000)
         })))

polys_heavy = (heavy.reduceToVectors(
            geometry=aoi, scale=scale, geometryType='polygon',
            labelProperty='heavy', maxPixels=1e13)
         .map(lambda f: f.set({
             'date': end_date,
             'class': 'heavy',
             'area_ha': f.geometry().area(1).divide(10000)
         })))

polys_all = polys.merge(polys_heavy)


In [ ]:
Map = geemap.Map(center=[19.31, 99.34], zoom=10)
Map.addLayer(dNBR, {'min':0, 'max':0.8, 'palette':['white','orange','red']}, 'dNBR')
Map.addLayer(burn,  {'palette':['#ffd400']}, 'Burn')
Map.addLayer(heavy, {'palette':['#ff00ff']}, 'Heavy')
Map.addLayerControl(); Map


Map(center=[19.31, 99.34], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

# Korat NDVI GIF

In [ ]:
!pip -q install earthengine-api geemap

from google.colab import auth
auth.authenticate_user()

import ee, geemap
ee.Authenticate()
ee.Initialize(project='*******')


In [ ]:
import ee

th_l1 = ee.FeatureCollection('FAO/GAUL/2015/level1') \
           .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))
korat  = th_l1.filter(ee.Filter.eq('ADM1_NAME', 'Nakhon Ratchasima')) \
              .geometry().dissolve()


In [ ]:
YEAR = 2024
STEP_DAYS = 10
MIN_NDVI, MAX_NDVI = 0.0, 0.8

def s2_clean(start, end, geom):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(geom)
           .filterDate(start, end))
    def mask_cloud(i):
        qa = i.select('QA60')
        cloud = qa.bitwiseAnd(1<<10).Or(qa.bitwiseAnd(1<<11))
        return i.updateMask(cloud.Not()).divide(10000)
    return s2.map(mask_cloud)

def ndvi(img):
    return img.normalizedDifference(['B8','B4']).rename('NDVI')

start = ee.Date.fromYMD(YEAR, 1, 1)
end   = ee.Date.fromYMD(YEAR+1, 1, 1)

n_steps = end.difference(start, 'day').divide(STEP_DAYS).floor()
date_seq = ee.List.sequence(0, n_steps.subtract(1))

def make_frame(i):
    i = ee.Number(i)
    sd = start.advance(i.multiply(STEP_DAYS), 'day')
    ed = sd.advance(STEP_DAYS, 'day')
    comp = s2_clean(sd, ed, korat).median().clip(korat)
    nd = ndvi(comp).updateMask(comp.select('B8').mask()) \
                   .set('system:time_start', sd.millis())
    return nd

ndvi_ic = ee.ImageCollection(date_seq.map(make_frame))


In [ ]:
Map = geemap.Map(center=[15.0, 102.1], zoom=7)
Map.addLayer(ndvi_ic.first(), {'min':MIN_NDVI, 'max':MAX_NDVI,
                               'palette':['#ffffe5','#f7fcb9','#d9f0a3','#addd8e',
                                          '#78c679','#41ab5d','#238443','#006837','#004529']},
             f'NDVI {YEAR} (first frame)')
Map.addLayer(korat, {}, 'Nakhon Ratchasima')
Map.addLayerControl(); Map


Map(center=[15.0, 102.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
def visualize(img):
    return img.visualize(min=MIN_NDVI, max=MAX_NDVI, palette=palette) \
              .set('system:time_start', img.get('system:time_start'))

vis_ic = ndvi_ic.map(visualize)

task = ee.batch.Export.video.toDrive(
    collection=vis_ic,
    description=f'NDVI_Korat_{YEAR}_video',
    folder='EarthEngine',
    fileNamePrefix=f'ndvi_korat_{YEAR}',
    framesPerSecond=6,
    region=korat,
    scale=100
)
task.start()

Export to Drive started. ไปดูความคืบหน้าใน Tasks แถบขวา (GEE)
